In [1]:
# -*- coding: utf-8 -*-
"""word_4_19.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1w-mu4xtSUIiUCcxbROUfA-sfj2-1ADTL
"""
# Commented out IPython magic to ensure Python compatibility.
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import math
import os
import cv2
import difflib 
import pandas as pd

units = 256
vocab_tar_size = 81
embedding_dim = 3 #embedding_dimensions =  number_of_categories**0.25
max_length=18
attention_features_shape = 91
img_height=48
img_width=192

def detect_contour(img):
    size=img.shape
    # print(size)
    x=0
    while not tf.reduce_any(tf.not_equal(img[:,x],1)):
        x+=5
    xmin=x-5 if x-5>0 else 0
    x=size[1]-1
    while not tf.reduce_any(tf.not_equal(img[:,x],1)):
        x-=5
    xmax=x+5 if x+5<size[1]-1 else size[1]-1
    
    y=0
    while not tf.reduce_any(tf.not_equal(img[y,:],1)):
        y+=7
    ymin=y-7 if y-7>0 else 0
    y=size[0]-1
    while not tf.reduce_any(tf.not_equal(img[y,:],1)):
        y-=7
    ymax=y+7 if y+7<size[0]-1 else size[0]-1
    
    return ymin,xmin,ymax,xmax

def slide_window(img):
    xstep=0
    img_patches=[]
    while xstep < img_width-10:
        img_patches.append(img[:,xstep:xstep+10])
        xstep+=2
    return img_patches

def precess_image(image):

    img = slant_correct(image)
    size = img.shape
    ymin,xmin,ymax,xmax=[0,0,size[0],size[1]]
    # width=0
    # img=tf.convert_to_tensor(img)
    if (ymax-ymin)<img_height:
        width=round((xmax-xmin)*(img_height/(ymax-ymin)))
    else:
        width=round((xmax-xmin)/((ymax-ymin)/img_height))
    if width>img_width:
        img = cv2.resize(img.numpy(),(img_width,img_height))
        width=img_width
    else:
        img = cv2.resize(img.numpy(),(width,img_height))
    
    # img = tf.cast(img,tf.float32)/255.
    # img=tf.where(img>0.88,tf.ones_like(img),img)
    # # img=1.-img
    img=tf.pad(img,[[0,0],[0,img_width-width]],constant_values=1)
    # plt.imshow(img,cmap='gray')
    # print(img.numpy())
    img_patches=slide_window(img)
    # print(img_patches[90].numpy())
    return img_patches

def show_patches(img):
    fig = plt.figure(figsize=(10, 10))
    i=0
    #sample_image=tf.reshape(sample_image,(y,x))
    for i in range(len(img)):
        ax = fig.add_subplot(10, 10, i+1)
        ax.set_title(str(i))
        ax.imshow(img[i],cmap='gray')
    plt.tight_layout()
    plt.show()

def slant_correct(gray):
#     img = cv2.imread(file_path)#,cv2.IMREAD_GRAYSCALE)
    
#     gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    ret, binary = cv2.threshold(gray,230,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    binary = 255 - binary
    contours, hierarchy = cv2.findContours(binary,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)

    sum_theta = 0
    sum_weight = 0
    for line in range(len(contours)):

#         cv2.drawContours(img,contours[line],-1,(0,0,255),3) 

        columns = []

        for i in range(1,len(contours[line])):
            columns.append(contours[line][i]-contours[line][i - 1])
        columns.append(contours[line][0]-contours[line][-1])

        a = []

        for i in range(len(columns)):
            if columns[i][0][0] == 0 and columns[i][0][1] == -1:
                a.append(6)
            elif columns[i][0][0] == 0 and columns[i][0][1] == 1:
                a.append(2)
            elif columns[i][0][0] == 1 and columns[i][0][1] == 1:
                a.append(1)
            elif columns[i][0][0] == 1 and columns[i][0][1] == 0:
                a.append(0)
            elif columns[i][0][0] == 1 and columns[i][0][1] == -1:
                a.append(7)
            elif columns[i][0][0] == -1 and columns[i][0][1] == 1:
                a.append(3)
            elif columns[i][0][0] == -1 and columns[i][0][1] == 0:
                a.append(4)
            elif columns[i][0][0] == -1 and columns[i][0][1] == -1:
                a.append(5)

        THRESH_V=1
        status=np.zeros_like(a)

        v_up = [2, 1, 0, 1, 2, 3, 4, 3]
        v_down = [2, 3, 4, 3, 2, 1, 0, 1]
        for i in range(len(a)):
            sum_slope_up = 0
            sum_slope_down = 0
            array=[]
            array.append(i-2)
            array.append(i-1)
            array.append(i)
            array.append(i+1 if i+1<len(a) else i+1-len(a))
            array.append(i+2 if i+2<len(a) else i+2-len(a))
            for j in array:
                sum_slope_up = sum_slope_up + v_up[a[j]]
                sum_slope_down = sum_slope_down + v_down[a[j]]
#             print(sum_slope_up,sum_slope_down)
#             if(sum_slope_up==sum_slope_down):status[i]=2
            if sum_slope_up < THRESH_V:
                status[i] = 1
            if sum_slope_down < THRESH_V:
                status[i] = -1
        cv2.waitKey(0)
#         print(status)
        p=contours[line][np.where(status==0)]

        try:
            for k in range(len(p)-1):

                dirt=p[k][0]-p[k+1][0]
                theta=-1*(dirt[1]/dirt[0])
                if theta>=0:
                    theta=theta if theta<1 else 1.
                else:
                    theta=theta if theta>-1 else -1.
                
                sum_theta+=theta*abs(dirt[1])
                sum_weight+=abs(dirt[1])

        except:
            pass
        
    h,w = gray.shape
    res = gray
    if sum_weight!=0:
        M=np.array([[1,sum_theta/sum_weight,0.5*w*abs(sum_theta/sum_weight)],[0,1,0]])
        res = cv2.warpAffine(gray, M,(w*2, h),flags=cv2.INTER_CUBIC,borderValue=(255))
        res=tf.cast(res,dtype=tf.float32)/255.
        res=tf.where(res>0.88,tf.ones_like(res),res)
        try:
            ymin,xmin,ymax,xmax = detect_contour(res)
            return res[ymin:ymax,xmin:xmax]
        except:
            return res
    else:
        res=tf.cast(res,dtype=tf.float32)/255.
        res=tf.where(res>0.88,tf.ones_like(res),res)
        return res


class Encoder(tf.keras.Model):
    def __init__(self, enc_units):
        super(Encoder, self).__init__()
        self.enc_units = enc_units
        self.conv1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(20,5,strides=1,
                                          activation='relu',
                                          padding='same',
                                          kernel_initializer='he_uniform'))
        self.maxpool1 = tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPool2D((2,2)))
        self.conv2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Conv2D(50,5,strides=1,
                                            activation='relu',
                                            padding='same',
                                            kernel_initializer='he_uniform'))
        self.maxpool2 = tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPool2D((2,2)))
        self.flatten = tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten())
        self.dense = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1024))
        self.dropout1 = tf.keras.layers.Dropout(0.5)
        
        self.LSTM = tf.keras.layers.LSTM(self.enc_units,  
                                         return_sequences=True)
        self.BLSTM = tf.keras.layers.Bidirectional(self.LSTM)
        self.dropout2 = tf.keras.layers.Dropout(0.5)
        self.LSTM2 = tf.keras.layers.LSTM(self.enc_units,
                                       return_sequences=True,
                                       return_state=True)
        self.BLSTM2=tf.keras.layers.Bidirectional(self.LSTM2)
        # self.LSTM3 = tf.kears,layers.LSTM(self.enc_units,
        #                                   return_sequences=True)
        # self.BLSTM3=tf.kears.layers.Bidirectional(self.LSTM3)

    def call(self, img, hidden):
        x=tf.expand_dims(img,-1)
        # print(batch_seq.shape)
        # print(x.shape)
        x=self.conv1(x)
        x=self.maxpool1(x)
        x=self.conv2(x)
        x=self.maxpool2(x) 
        x=self.flatten(x)
        x=self.dense(x)
        x=self.dropout1(x)
        output = self.BLSTM(x)#h短期记忆,c长期记忆
        output = self.dropout2(output)
        # output = self.BLSTM3(output)
        output, state_h, state_c, b_state_h, b_state_c = self.BLSTM2(output)
        # output, state_h, state_c = self.BLSTM2(output)
        return output, [tf.concat([state_h,b_state_h],axis=-1),tf.concat([state_c,b_state_c],axis=-1)]
        # return output, [state_h,state_c]


    def initialize_hidden_state(self,batch_sz):
        return tf.zeros((batch_sz, self.enc_units))


class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        # 隐藏层的形状 == （批大小，隐藏层大小）
        # hidden_with_time_axis 的形状 == （批大小，1，隐藏层大小）
        # 这样做是为了执行加法以计算分数  
        hidden_with_time_axis = tf.expand_dims(hidden, 1)

        # 分数的形状 == （批大小，最大长度，1）
        # 我们在最后一个轴上得到 1， 因为我们把分数应用于 self.V
        # 在应用 self.V 之前，张量的形状是（批大小，最大长度，单位）
        score = self.V(tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis)))

        # 注意力权重 （attention_weights） 的形状 == （批大小，最大长度，1）
        attention_weights = tf.nn.softmax(score, axis=1)

        # 上下文向量 （context_vector） 求和之后的形状 == （批大小，隐藏层大小）
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units):
        super(Decoder, self).__init__()
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(self.dec_units, 
                                       return_sequences=True,
                                       return_state=True)
                                      
        self.fc1 = tf.keras.layers.Dense(vocab_size)

        # 用于注意力
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, enc_output, hidden):
        # 编码器输出 （enc_output） 的形状 == （批大小，最大长度，隐藏层大小）
        context_vector, attention_weights = self.attention(enc_output, hidden[1])
        # x 在通过嵌入层后的形状 == （批大小，1，嵌入维度）
        x = self.embedding(x)

        # x 在拼接 （concatenation） 后的形状 == （批大小，1，嵌入维度 + 隐藏层大小）
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        # x = tf.expand_dims(x,1)
        # 将合并后的向量传送到 GRU
        output, state_h, state_c = self.lstm(x,hidden)

        output = tf.concat([tf.expand_dims(context_vector, 1), output], axis=-1)

        output = self.fc1(output)
        # 输出的形状 == （批大小 * 1，隐藏层大小）
        output = tf.reshape(output, (-1, output.shape[2]))

        return output, [state_h,state_c], attention_weights

def evaluate(image):
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = encoder.initialize_hidden_state(1)
    temp_input = tf.expand_dims(tf.convert_to_tensor(precess_image(image)), 0)
    # print(temp_input.shape)
    features,hidden = encoder(temp_input,hidden)

    dec_input = tf.expand_dims([all_label_lang.word_index['@']], 0)
    # print(dec_input)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)

        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

        predicted_id = tf.argmax(predictions, 1).numpy()[0]
        # predicted_id=tf.random.categorical(predictions,1)[0][0].numpy()
        # print(predicted_id)
        result.append(all_label_lang.index_word[predicted_id])
        
        if all_label_lang.index_word[predicted_id] == '^':
            return result, attention_plot
        
        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot

def cut_word(img):
    ymin,xmin,ymax,xmax = detect_contour(img)
    oimg = img[ymin:ymax,xmin:xmax]

    kernel = np.ones((5,17),np.uint8)
    img = cv2.erode(oimg,kernel)
    img=cv2.GaussianBlur(img,(15,15),100)
    img = tf.cast(img,tf.float32)/255.
    img = 1 - img
    y = tf.reduce_sum(img,axis=0)
    x = np.arange(0,int(y.shape[0]),1)

    indexs=tf.where(tf.equal(y,tf.reduce_min(y)))
    pre=0
    res=[]

    for i in range(1,indexs.shape[0]):
        if indexs[i]-indexs[i-1]>1:
            res.extend((indexs[pre].numpy()+indexs[i-1].numpy())//2)
            pre=i
    res.extend((indexs[-1].numpy()+indexs[pre].numpy())//2)
    res.append(img.shape[1])
    pre=0
    result=[]
    for i in range(len(res)):
        result.append(oimg[:,pre:res[i]])
        pre=res[i]
    return result

def minDistance(word1, word2):
        n1 = len(word1)
        n2 = len(word2)
        dp = [[0] * (n2 + 1) for _ in range(n1 + 1)]
        # 第一行
        for j in range(1, n2 + 1):
            dp[0][j] = dp[0][j-1] + 1
        # 第一列
        for i in range(1, n1 + 1):
            dp[i][0] = dp[i-1][0] + 1
        for i in range(1, n1 + 1):
            for j in range(1, n2 + 1):
                if word1[i-1] == word2[j-1]:
                    dp[i][j] = dp[i-1][j-1]
                else:
                    dp[i][j] = min(dp[i][j-1], dp[i-1][j], dp[i-1][j-1] ) + 1
        #print(dp)      
        return dp[-1][-1]

def lexicon(a):
    da=pd.read_csv("C:/Users/sss/Desktop/spell.csv")
    p=da['WORD'].values
    max_ratio=10
    max_index=0

    for t in range(len(p)):
        tmp=minDistance(a,p[t])
        if tmp<=max_ratio:
            max_index=t
            max_ratio=tmp

        if max_ratio==0:
            return p[max_index]
        
    return p[max_index]

In [2]:
from PyQt5 import QtCore,QtGui,QtWidgets
from PyQt5.QtWidgets import *
import sys
import qtawesome
import os
 
class MainUi(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()
        self.init_ui()
        self.init_style()
        self.setWindowTitle("识别程序")
        self.cwd = os.getcwd() # 获取当前程序文件位置
        self.model = 0
#         self.setWindowOpacity(0.9) # 设置窗口透明度
#         self.setAttribute(QtCore.Qt.WA_TranslucentBackground) # 设置窗口背景透明
        self.setWindowFlag(QtCore.Qt.FramelessWindowHint) # 隐藏边框
        self.main_layout.setSpacing(0)
        self.img_path=''
        

    def init_ui(self):
        self.setFixedSize(960,700)
        self.main_widget = QtWidgets.QWidget()  # 创建窗口主部件
        self.main_layout = QtWidgets.QGridLayout()  # 创建主部件的网格布局
        self.main_widget.setLayout(self.main_layout)  # 设置窗口主部件布局为网格布局
 
        self.left_widget = QtWidgets.QWidget()  # 创建左侧部件
        self.left_widget.setObjectName('left_widget')
        self.left_layout = QtWidgets.QGridLayout()  # 创建左侧部件的网格布局层
        self.left_widget.setLayout(self.left_layout) # 设置左侧部件布局为网格
 
        self.right_widget = QtWidgets.QWidget() # 创建右侧部件
        self.right_widget.setObjectName('right_widget')
        self.right_layout = QtWidgets.QGridLayout()
        self.right_widget.setLayout(self.right_layout) # 设置右侧部件布局为网格
 
        self.main_layout.addWidget(self.left_widget,0,0,12,2) # 左侧部件在第0行第0列，占8行3列
        self.main_layout.addWidget(self.right_widget,0,2,12,10) # 右侧部件在第0行第3列，占8行9列
        self.setCentralWidget(self.main_widget) # 设置窗口主部件
        
        self.left_ocr = QtWidgets.QPushButton("识别") # 关闭按钮
        self.left_close = QtWidgets.QPushButton("") # 空白按钮
        self.left_mini = QtWidgets.QPushButton("")  # 最小化按钮

        self.left_button_1 = QtWidgets.QPushButton(qtawesome.icon('fa.music',color='white'),"文本行识别")
        self.left_button_1.setObjectName('left_button')
        self.left_button_2 = QtWidgets.QPushButton(qtawesome.icon('fa.sellsy',color='white'),"单词识别")
        self.left_button_2.setObjectName('left_button')
        
        self.left_button_1.clicked.connect(self.line_ocr)
        self.left_button_2.clicked.connect(self.word_ocr)
        self.left_ocr.clicked.connect(self.do_ocr)
        self.left_layout.addWidget(self.left_mini, 0, 0,1,1)
        self.left_layout.addWidget(self.left_ocr, 0, 2,1,1)
        self.left_layout.addWidget(self.left_close, 0, 1, 1, 1)
        self.left_layout.addWidget(self.left_button_1, 1, 0,1,3)
        self.left_layout.addWidget(self.left_button_2, 2, 0,1,3)
        
        self.right_bar_widget = QtWidgets.QWidget() # 右侧顶部搜索框部件
        self.right_bar_layout = QtWidgets.QGridLayout() # 右侧顶部搜索框网格布局
        self.right_bar_widget.setLayout(self.right_bar_layout)
        
        self.search_icon = QtWidgets.QPushButton(chr(0xf002) + ' '+'选择图片')
        self.search_icon.setFont(qtawesome.font('fa', 16))
        self.right_bar_widget_search_input = QtWidgets.QLineEdit()
#         self.right_bar_widget_search_input.setReadOnly(True)
        self.right_bar_widget_search_input.setPlaceholderText("图片路径")

        self.search_icon.clicked.connect(self.slot_btn_chooseFile)
        self.right_bar_layout.addWidget(self.search_icon,0,0,1,1)
        self.right_bar_layout.addWidget(self.right_bar_widget_search_input,0,1,1,8)

        self.right_layout.addWidget(self.right_bar_widget, 0, 0, 1, 9)
        
        self.right_picture_label = QtWidgets.QLabel("选取图片")
        self.right_picture_label.setObjectName('right_picture')
        self.right_picture = QtWidgets.QLabel("")
#         self.right_picture.setFixedSize(300, 200)
        self.right_result_label = QtWidgets.QLabel("识别结果")
        self.right_result_label.setObjectName('right_label')
        self.right_result = QtWidgets.QLabel("")

        
        self.right_layout.addWidget(self.right_picture_label, 1, 0, 1, 9)
        self.right_layout.addWidget(self.right_result_label, 3, 0, 1, 9)
        self.right_layout.addWidget(self.right_picture, 2, 0, 1, 9)
        self.right_layout.addWidget(self.right_result, 4, 0, 1, 9)
        
    def init_style(self):
        self.left_ocr.setStyleSheet('''QPushButton{background:#F76677;border-radius:5px;}QPushButton:hover{background:red;}''')
        self.left_close.setStyleSheet('''QPushButton{background:#F7D674;border-radius:5px;}QPushButton:hover{background:yellow;}''')
        self.left_mini.setStyleSheet('''QPushButton{background:#6DDF6D;border-radius:5px;}QPushButton:hover{background:green;}''')
        
        self.left_widget.setStyleSheet('''
            QPushButton{border:none;color:white;}
            QPushButton#left_label{
                border:none;
                border-bottom:1px solid white;
                font-size:18px;
                font-weight:700;
                font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
            }
            QPushButton#left_button:hover{border-left:4px solid red;font-weight:700;}
            QWidget#left_widget{
                    background:gray;
                    border-top:1px solid white;
                    border-bottom:1px solid white;
                    border-left:1px solid white;
                    border-top-left-radius:10px;
                    border-bottom-left-radius:10px;
                }
        ''')
 
        
        self.right_bar_widget_search_input.setStyleSheet(
            '''QLineEdit{
                    border:1px solid gray;
                    width:300px;
                    border-radius:10px;
                    padding:2px 4px;
            }''')
        self.right_widget.setStyleSheet('''
            QWidget#right_widget{
                color:#232C51;
                background:white;
                border-top:1px solid darkGray;
                border-bottom:1px solid darkGray;
                border-right:1px solid darkGray;
                border-top-right-radius:10px;
                border-bottom-right-radius:10px;
            }
            QLabel#right_label{
                border:none;
                font-size:16px;
                font-weight:700;
                font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
            }
        ''')
 
        
    def slot_btn_chooseFile(self):
        fileName_choose, filetype = QFileDialog.getOpenFileName(self,  
                                    "选取文件",  
                                    self.cwd, # 起始路径 
                                    "All Files (*);;JPEG Files (*.jpg);;PNG Files (*.png);;TIF Files (*.tif)")   # 设置文件扩展名过滤,用双分号间隔

        if fileName_choose == "":
            return

        self.right_bar_widget_search_input.setText(fileName_choose)
        jpg = QtGui.QPixmap(fileName_choose)
        w=jpg.width()
        h=jpg.height()
        if w>=700:
            jpg=jpg.scaled(700,round(h*700/w))
        self.right_picture.setPixmap(jpg)
        self.img_path=fileName_choose

    def do_ocr(self):
        img = cv2.imread(self.img_path,cv2.IMREAD_GRAYSCALE)
        if self.model==0:
            re=''
            rimg=cut_word(img)
            for tmp in rimg:
                re=re+''.join(evaluate(tmp)[0][:-1])+" "     
                self.right_result.setText(re)
        else:
            result=''.join(evaluate(img)[0][:-1])
            self.right_result.setText(result)
                                      
    def line_ocr(self):
        self.model=0
    def word_ocr(self):
        self.model=1
 
def main():
    app = QtWidgets.QApplication(sys.argv)
    gui = MainUi()
    gui.show()
    sys.exit(app.exec_())
 
if __name__ == '__main__':
    all_label_lang=np.load("./training_checkpoints/lang.npy",allow_pickle=True).item()

    encoder = Encoder(units)
    decoder = Decoder(vocab_tar_size, embedding_dim, units*2)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001,decay=0.02)

    checkpoint_path = './training_checkpoints/81_train'
    # checkpoint_path = './training_checkpoints/train10000'
    ckpt = tf.train.Checkpoint(encoder=encoder,
                               decoder=decoder,
                               optimizer = optimizer)
    ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=2)

    start_epoch = 0
    if ckpt_manager.latest_checkpoint:
        start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
        # restoring the latest checkpoint in checkpoint_path
        ckpt.restore(ckpt_manager.latest_checkpoint)
    main()

E:\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:168: RuntimeWarning: divide by zero encountered in int_scalars


SystemExit: 0

E:\Python\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3333: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
# class TrieNode(object):
#     def __init__(self, value=None, count=0, parent=None):
#         # 值
#         self.value = value
#         # 频数统计
#         self.count = count
#         # 父结点
#         self.parent = parent
#         # 子节点，{value:TrieNode}
#         self.children = {}


# class Trie(object):
#     def __init__(self):
#         # 创建空的根节点
#         self.root = TrieNode()
#         self.lexicon=[]

#     def insert(self, sequence):
#         """
#         基操，插入一个序列
#         :param sequence: 字符串
#         :return:
#         """
#         cur_node = self.root
#         self.lexicon.append(sequence)
#         for item in sequence:
#             if item not in cur_node.children:
#                 # 插入结点
#                 child = TrieNode(value=item, count=1, parent=cur_node)
#                 cur_node.children[item] = child
#                 cur_node = child
#             else:
#                 # 更新结点
#                 cur_node = cur_node.children[item]
#                 cur_node.count += 1

#     def search(self, sequence):
#         """
#         基操，查询是否存在完整序列
#         :param sequence: 字符串
#         :return:
#         """
#         cur_node = self.root
# #         result=[]
#         mark = True
#         i=0
#         for i in range(len(sequence)):
#             item = sequence[i]
#             print(self.lexicon)
#             if item not in cur_node.children:
#                 mark = False
#                 print(item)
#                 return self.min_distance(sequence,i)
#             else:
# #                 result.append(cur_node.value)
#                 cur_node = cur_node.children[item]
#         # 如果还有子结点，说明序列并非完整
#         if cur_node.children:
#             mark = False
#         return self.min_distance(sequence,i)
    
#     def min_distance(self,sequence,i):
#         mdis=5
#         mw=sequence
#         print(i)
#         for w in self.lexicon:
#             if i==0 or w.startswith(sequence[:i]):
#                 print(w)
#                 tmp = minDistance(sequence,w)
#                 if mdis > tmp:
#                     mdis = tmp
#                     mw = w
#         return mw

In [ ]:
!pyinstall -F 